# Glucose Control in Patients With Type I Diabetes
$\hspace{10mm}$Lizett Pink, Andrew Spitzer, Travis Tredway & Kyle Vachon

$\hspace{10mm}$Prof. Jeffrey C Kantor

$\hspace{10mm}$University of Notre Dame

$\hspace{10mm}$Department of Chemical and Biomolecular Engineering

$\hspace{10mm}$Chemical Process Control

$\hspace{10mm}$4 May 2017

## Contents

* [1. Problem Statement](#Section00)
* [2. Theoretical Development](#Section01)
     * [A. Governing Equations](#Section02)
     * [B. Block Diagram](#Section03)
     * [C. PID Control](#Section04)
* [3. Positive Disturbance: Exposure to a Meal](#Section05)
     * [A. Initial Conditions - Varied Insulin Input Rate by Controller](#Section06)
     * [B. Closer Look at Ideal Insulin Input Rate](#Section07)
     * [C. Fixed Time Grid Simulation](#Section08)
     * [D. PI Controlled Insulin Input Rate](#Section09)
* [4. Negative Disturbance: Exposure to Exercise](#Section10)
     * [A. Fixed Time Grid Simulation](#Section11)
     * [B. PI Controlled Insulin Input Rate](#Section12)
* [5. Combined Disturbances: Snack, Exercise, Meal](#Section13)
     * [A. Fixed Time Grid Simulation](#Section14)
     * [B. PI Controlled Insulin Input Rate](#Section15)
     * [C. Interactive PID Controlled Simulation](#Section16)
* [6. Conclusion](#Section17)